In [ ]:
# Install any necessary library for this Machine Learning Model
import requests
import pandas as pd
import numpy as np
from hmmlearn import hmm
from sklearn.preprocessing import MinMaxScaler

In [ ]:
class DataRetriever: #class to handle data retrieving using API key and data storing

    def __init__(self, api_key, base_url="https://api.datasource.cybotrade.rs"):

        self.api_key = api_key
        self.base_url = base_url
        self.headers = {
            "X-API-KEY": self.api_key
        }

    def fetch_data_and_save_csv(self, endpoint, params, csv_filename):
        # conenct to server and save the retrieved data into a csv file
        url = f"{self.base_url}{endpoint}"
        try:
            response = requests.get(url, headers=self.headers, params=params)
            response.raise_for_status()  # Raise an exception for bad status codes
            json_data = response.json()
            if 'data' in json_data:
                df = pd.DataFrame(json_data['data'])
                try:
                    df.to_csv(csv_filename, index=False)
                    print(f"Data successfully saved to {csv_filename}")
                    return True
                except Exception as e:
                    print(f"Error saving to CSV: {e}")
                    return False
            else:
                print(f"Error: 'data' key not found in the response for endpoint: {endpoint}")
                print(json_data)
                return False
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data from {url}: {e}")
            return False

In [ ]:
class DataProcessor:

  def __init__(self, csv_filepath):
      self.csv_filepath = csv_filepath
      self.df = None

  def load_data(self): # function that reads the csv file and load data into df
        try:
            df = pd.read_csv(self.csv_filepath)
            print(f"Successfully loaded data from {self.csv_filepath}")
            print(f"Initial DataFrame shape: {df.shape}")
            print(df.head())
            print(df.info())
            self.df = df
            return df
        except FileNotFoundError:
            print(f"Error: The file '{self.csv_filepath}' was not found.")
            return None

  def clean_data(self): # function to handle data conversion or data cleaning such as checking for missing value 
        if self.df is None:
            print("Error: No data loaded. Cannot clean.")
            return None

        print("\n--- Checking for Missing Values ---")
        print(self.df.isnull().sum())

        # Handle missing values (replace with your strategy)
        self.df.fillna(0, inplace=True)
        print("\nMissing Values After Handling:")
        print(self.df.isnull().sum())

        print("\n--- Data Type Conversion ---")
        if 'time' in self.df.columns:  # Check for 'time' column
            try:
                self.df['timestamp'] = pd.to_datetime(self.df['time'], unit='s')  # Assuming 'time' is in seconds
                self.df.drop('time', axis=1, inplace=True)
                print("'time' converted to 'timestamp'.")
            except Exception as e:
                print(f"Error converting 'time': {e}")
        elif 'start_time' in self.df.columns:  # Fallback for 'start_time' if it exists
            try:
                self.df['timestamp'] = pd.to_datetime(self.df['start_time'], unit='ms')
                self.df.drop('start_time', axis=1, inplace=True)
                print("'start_time' converted to 'timestamp'.")
            except Exception as e:
                print(f"Error converting 'start_time': {e}")
        else:
            print("Warning: Neither 'start_time' nor 'time' column found for timestamp conversion.")

        print("\n--- Handling Duplicate Data ---")
        print(f"Number of duplicate rows: {self.df.duplicated().sum()}")
        self.df.drop_duplicates(inplace=True)
        print(f"Number of rows after removing duplicates: {self.df.shape[0]}")

        return self.df

  def preprocess_data(self): # function that calculate and add features to dataframe

        if self.df is None:
            print("Error: No data loaded. Cannot preprocess.")
            return None

        print("\n--- Feature Engineering ---")
        if 'bidsAmount' in self.df.columns and 'asksAmount' in self.df.columns: # calculate bid to ask ratio
            self.df['bids_asks_ratio_amount'] = self.df['bidsAmount'] / (self.df['asksAmount'] + 1e-8)
            print("Added 'bids_asks_ratio_amount' feature.")

        if 'bidsUsd' in self.df.columns and 'asksUsd' in self.df.columns: # calculate bid (USD) to ask (USD) ratio
            self.df['bids_asks_ratio_usd'] = self.df['bidsUsd'] / (self.df['asksUsd'] + 1e-8)
            print("Added 'bids_asks_ratio_usd' feature.")

        print("\n--- Sorting and Indexing ---")
        if 'timestamp' in self.df.columns:
            self.df.sort_values('timestamp', inplace=True)
            self.df.set_index('timestamp', inplace=True)
            print("Sorted by 'timestamp' and set as index.")
        else:
            print("Warning: 'timestamp' column not found.")

        return self.df

In [ ]:
class HMMTrader: # class to mainly handle HMM Model Training
    """
        Trains the Hidden Markov Model.

        Args:
            df (pd.DataFrame): The processed DataFrame containing the features.
            feature_columns (list): A list of column names to use as features for the HMM.

        Returns:
            pd.DataFrame: The DataFrame with the predicted HMM states added.
    """

    def __init__(self, n_components=5):

        self.n_components = n_components
        self.model = None
        self.scaler = None
        self.feature_columns = None

    def train_hmm(self, df, feature_columns):

        if df is None or df.empty or not all(col in df.columns for col in feature_columns):
            print("Error: Invalid DataFrame or missing feature columns for HMM training.")
            return df

        self.feature_columns = feature_columns
        data = df[self.feature_columns].values

        # Scale the data
        self.scaler = MinMaxScaler()
        scaled_data = self.scaler.fit_transform(data)

        # Train the HMM
        self.model = hmm.GaussianHMM(n_components=self.n_components, covariance_type="full", n_iter=1000, random_state=42) # Added random_state for reproducibility
        self.model.fit(scaled_data)

        # Predict the hidden states
        hidden_states = self.model.predict(scaled_data)
        df['hmm_state'] = hidden_states

        print("\nHMM model trained successfully. Predicted states added to DataFrame.")
        return df

    def save_state_to_csv(self, df_with_states, filename="hmm_states.csv"):

        if df_with_states is not None and 'hmm_state' in df_with_states.columns:
            try:
                df_with_states.to_csv(filename, index=True)  # Include index (timestamp if set)
                print(f"HMM states saved to {filename}")
            except Exception as e:
                print(f"Error saving HMM states to CSV: {e}")
        else:
            print("Error: DataFrame with HMM states is None or 'hmm_state' column not found.")

In [ ]:
class MainApp: # class to handle operations such as calling functions from other classes
    def __init__(self, api_key, csv_filepath, n_hmm_components=3, hmm_features=['bids_asks_ratio_amount', 'bids_asks_ratio_usd']):
        self.data_retriever = DataRetriever(api_key)
        self.data_processor = DataProcessor(csv_filepath)
        self.hmm_trader = HMMTrader(n_components=n_hmm_components)  # Instantiate HMMTrader
        self.hmm_features = hmm_features  # Define the features for HMM
        self.trained_df_with_states = None

    def run(self):
        endpoint = "/coinglass/spot/orderbook/history"
        params = {
            "exchange": "Binance",
            "symbol": "BTCUSDT",
            "interval": "1h",
            "start_time": "20191001T100000",
            "limit": 10000
        }
        csv_filename = "coinglass_orderbook_history.csv"
        hmm_output_filename = "hmm_states_output.csv"

        # 1. Retrieve data and save to CSV
        if self.data_retriever.fetch_data_and_save_csv(endpoint, params, csv_filename):
            # 2. Load data from the CSV
            self.data_processor.load_data()
            cleaned_df = self.data_processor.clean_data()
            if cleaned_df is not None:
                # 3. Preprocess the cleaned data
                processed_df = self.data_processor.preprocess_data()
                if processed_df is not None:
                    print("\nFinal processed DataFrame:")
                    print(processed_df.head())
                    # 4. Train HMM model
                    self.trained_df_with_states = self.hmm_trader.train_hmm(processed_df.copy(), self.hmm_features)
                    if self.trained_df_with_states is not None:
                        print("\nDataFrame with HMM states:")
                        print(self.trained_df_with_states.reset_index()[['timestamp'] + self.hmm_features + ['hmm_state']].head())
                        # 5. Save States to csv
                        self.hmm_trader.save_state_to_csv(self.trained_df_with_states, hmm_output_filename)
                    else:
                        print("HMM training failed.")
                else:
                    print("Data preprocessing failed. Cannot proceed with HMM training.")
            else:
                print("Data cleaning failed. Cannot proceed with processing.")
        else:
            print("Data retrieval failed. Cannot proceed with data loading.")

In [ ]:
if __name__ == "__main__": # passing api key and csv filename from main function
    api_key = "MKkf7j3pCKwrMOENp4JNKXWGK5JzYqI18jFKFcBUgxOMfHke"
    csv_filepath = "coinglass_orderbook_history.csv"
    app = MainApp(api_key, csv_filepath)
    app.run()